In [1]:
import pandas as pd
data=pd.read_csv('apple_stock_data.csv')

In [2]:
data.head()

,Date,Close/Last,Volume,Open,High,Low
0,02/28/2020,$273.36,106721200,$257.26,$278.41,$256.37
1,02/27/2020,$273.52,80151380,$281.1,$286,$272.96
2,02/26/2020,$292.65,49678430,$286.53,$297.88,$286.5
3,02/25/2020,$288.08,57668360,$300.95,$302.53,$286.13
4,02/24/2020,$298.18,55548830,$297.26,$304.18,$289.23


In [3]:
## convert date column to datetime using pandas and set as index
data['Date']=pd.to_datetime(data['Date'])
data.set_index('Date',inplace=True)


In [4]:
data.head()

,Close/Last,Volume,Open,High,Low
Date,,,,,
2020-02-28,$273.36,106721200,$257.26,$278.41,$256.37
2020-02-27,$273.52,80151380,$281.1,$286,$272.96
2020-02-26,$292.65,49678430,$286.53,$297.88,$286.5
2020-02-25,$288.08,57668360,$300.95,$302.53,$286.13
2020-02-24,$298.18,55548830,$297.26,$304.18,$289.23


In [18]:
## select the close column and set it as a dataframe,
## using double brackets ensures it returns a dataframe
print(data.columns)

data.columns=data.columns.str.strip()
data=data[['Close/Last']]

Index(['Close/Last'], dtype='object')


In [10]:
data.head()

,Close/Last
Date,
2020-02-28,$273.36
2020-02-27,$273.52
2020-02-26,$292.65
2020-02-25,$288.08
2020-02-24,$298.18


In [12]:
### we will be using lstm and linear regression models
### lstms ability can model comples time dependant patterns
### while linear regression can identify and follow broader trends

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2518 entries, 2020-02-28 to 2010-03-01
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Close/Last  2518 non-null   object
dtypes: object(1)
memory usage: 39.3+ KB


In [21]:
##clean the  close column
data['Close/Last']=data[['Close/Last']].replace('[\$,]','',regex=True).astype(float)

<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
C:\Users\Admin\AppData\Local\Temp\ipykernel_11504\3665659768.py:2: SyntaxWarning: invalid escape sequence '\$'
  data['Close/Last']=data[['Close/Last']].replace('[\$,]','',regex=True).astype(float)


In [22]:
#using minmaxscaler , to scale between 0 and 1
from sklearn.preprocessing import MinMaxScaler 

#

scaler=MinMaxScaler(feature_range=(0,1))
data['Close/Last']=scaler.fit_transform(data[['Close/Last']])

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2518 entries, 2020-02-28 to 2010-03-01
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Close/Last  2518 non-null   float64
dtypes: float64(1)
memory usage: 39.3 KB


In [24]:
data.head()

,Close/Last
Date,
2020-02-28,0.818943
2020-02-27,0.819481
2020-02-26,0.883813
2020-02-25,0.868444
2020-02-24,0.902409


In [26]:
import numpy as np 

def create_sequences(data,seq_length=60):
    X,y=[],[]
    for i in range(len(data)-seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])

    return np.array(X),np.array(y)

X,y=create_sequences(data['Close/Last'].values)

In [28]:
##  split data into training and testing

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=42)

In [32]:
len(X_train),len(y_train),X_train.shape

(1646, 1646, (1646, 60))

In [30]:
len(X_test),len(y_test)

(812, 812)

In [31]:
## building the lstm model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [33]:
lstm_model=Sequential([
    LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],1)),
    LSTM(units=50),
    Dense(1)
])

c:\Users\Admin\.conda\envs\hybrid\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [34]:
## compile the model
lstm_model.compile(optimizer='adam',loss='mean_squared_error')
lstm_model.fit(X_train,y_train,epochs=20,batch_size=32)

Epoch 1/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 9s 64ms/step - loss: 0.0083
Epoch 2/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - loss: 3.0241e-04
Epoch 3/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - loss: 2.5303e-04
Epoch 4/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - loss: 2.6161e-04
Epoch 5/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - loss: 2.3591e-04
Epoch 6/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - loss: 2.2762e-04
Epoch 7/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - loss: 2.1981e-04
Epoch 8/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - loss: 2.1058e-04
Epoch 9/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - loss: 2.2473e-04
Epoch 10/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - loss: 2.0217e-04
Epoch 11/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - loss: 1.9572e-04
Epoch 12/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 70ms/step - loss: 1.9662e-04
Epoch 13/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - loss: 1.8129e-04
Epoch 14/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 70ms/step - loss: 1.8836e-04
Epoch 15/20
52/52 ━